# [SOLUTION] Exercise - Building an AI Agent with Tools

In this exercise, you'll build an AI agent that can use tools to enhance its capabilities. You'll learn how to create an agent that can understand when to use tools, process their results, and maintain a coherent conversation.

## Challenge

Imagine you're building a smart coding assistant that needs to:
- Answer programming questions
- Execute code snippets
- Look up documentation
- Perform calculations
- Search through codebases

Instead of hard-coding when to use each capability, your agent should intelligently decide when and how to use its available tools.

## Setup
First, let's import the necessary libraries:

In [1]:
from typing import List, Any
from dotenv import load_dotenv
import json

from lib.messages import UserMessage, SystemMessage, ToolMessage
from lib.tooling import tool
from lib.llm import LLM

## Understanding the Components

Before we build our agent, let's understand the key components we'll be working with:

- `LLM`: The language model wrapper that handles tool execution
- `SystemMessage`: Defines the agent's role and behavior
- `UserMessage`: Represents user inputs
- `ToolMessage`: Contains tool execution results
- `tool`: Decorator for creating tools

## Building the Agent Class

Your task is to create an Agent class that can:
1. Initialize with a specific role and set of tools
2. Process user messages
3. Decide when to use tools
4. Handle tool responses

In [2]:
class Agent:
    """An AI Agent that can use tools to help answer questions"""
    
    def __init__(
        self,
        role: str = "Personal Assistant",
        instructions: str = "Help users with any question",
        model: str = "gpt-4o-mini",
        temperature: float = 0.0,
        tools: List[Any] = None
    ):
        """Initialize the agent with its configuration and tools
        
        Args:
            role: The agent's role/persona
            instructions: Basic instructions for the agent
            model: The LLM model to use
            temperature: Creativity parameter (0.0 = more 'deterministic')
            tools: List of tools the agent can use
        """
        self.model = model
        self.role = role
        self.instructions = instructions
        self.tools = tools

        # Load environment variables (e.g. API keys)
        load_dotenv()
        
        # Initialize the LLM with tools if provided
        self.llm = LLM(
            model=model,
            temperature=temperature,
            tools=tools,
        )

    def invoke(self, user_message: str) -> str:
        """Process a user message and return a response
        
        Args:
            user_message: The user's input message
            
        Returns:
            The agent's response after processing tools if needed
        """
        messages = [
            SystemMessage(
                content=(
                    f"You're an AI Agent and your role is {self.role}. "  
                    f"Your instructions: {self.instructions}"
                )
            )
        ]
        # Add user message to conversation
        messages.append(UserMessage(content=user_message))
        
        # Get AI response and add to conversation
        ai_message = self.llm.invoke(messages)
        messages.append(ai_message)

        # Check if tools were required
        while ai_message.tool_calls:
            # Process each tool call                    
            for call in ai_message.tool_calls:
                # Access tool call data correctly
                function_name = call.function.name
                function_args = json.loads(call.function.arguments)
                tool_call_id = call.id
                # Find the matching tool
                tool = next((t for t in self.tools if t.name == function_name), None)
                if tool:
                    result = tool(**function_args)
                    messages.append(
                        ToolMessage(
                            content=json.dumps(result), 
                            tool_call_id=tool_call_id, 
                            name=function_name, 
                        )
                    )
                
            # Get final AI response after tool usage and add to conversation
            ai_message = self.llm.invoke(messages)
            messages.append(ai_message)

        for m in messages:
            print(m)
        return ai_message.content


## Testing Your Agent

Once you've implemented the Agent class, test it with different scenarios:

1. Basic conversation without tools

In [3]:
agent = Agent(role="Coding Assistant")
response = agent.invoke("What is Python? Be concise")
print(response)

content="You're an AI Agent and your role is Coding Assistant. Your instructions: Help users with any question" role='system'
content='What is Python? Be concise' role='user'
content='Python is a high-level, interpreted programming language known for its readability and simplicity. It supports multiple programming paradigms, including procedural, object-oriented, and functional programming. Python is widely used for web development, data analysis, artificial intelligence, scientific computing, and automation, among other applications.' role='assistant' tool_calls=None
Python is a high-level, interpreted programming language known for its readability and simplicity. It supports multiple programming paradigms, including procedural, object-oriented, and functional programming. Python is widely used for web development, data analysis, artificial intelligence, scientific computing, and automation, among other applications.


2. Create a calculator tool

In [4]:
@tool
def calculate(expression: str) -> float:
    """Evaluate a mathematical expression"""
    return eval(expression)

In [5]:
# Create an agent with the calculator tool
math_agent = Agent(
    role="Math Assistant",
    tools=[calculate]
)

OpenAIError: The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable

In [6]:
response = math_agent.invoke("What is 23 * 45?")
print(response)

content="You're an AI Agent and your role is Math Assistant. Your instructions: Help users with any question" role='system'
content='What is 23 * 45?' role='user'
content=None role='assistant' tool_calls=[ChatCompletionMessageToolCall(id='call_klzgArJHewvSYoIzGo8IQdJs', function=Function(arguments='{"expression":"23 * 45"}', name='calculate'), type='function')]
content='1035' role='tool' tool_call_id='call_klzgArJHewvSYoIzGo8IQdJs' name='calculate'
content='The result of \\( 23 \\times 45 \\) is 1035.' role='assistant' tool_calls=None
The result of \( 23 \times 45 \) is 1035.


In [7]:
# Test multiple tool usage
response = math_agent.invoke("If I multiply 3 by 5, what do I get? Then later add 7")
print(response)

content="You're an AI Agent and your role is Math Assistant. Your instructions: Help users with any question" role='system'
content='If I multiply 3 by 5, what do I get? Then later add 7' role='user'
content=None role='assistant' tool_calls=[ChatCompletionMessageToolCall(id='call_ng19o6ZlDmgnlLdU2m3hyqSs', function=Function(arguments='{"expression": "3 * 5"}', name='calculate'), type='function'), ChatCompletionMessageToolCall(id='call_EVP76gLh7JUWys9uQqyRYQ3c', function=Function(arguments='{"expression": "3 * 5 + 7"}', name='calculate'), type='function')]
content='15' role='tool' tool_call_id='call_ng19o6ZlDmgnlLdU2m3hyqSs' name='calculate'
content='22' role='tool' tool_call_id='call_EVP76gLh7JUWys9uQqyRYQ3c' name='calculate'
content='If you multiply 3 by 5, you get 15. If you then add 7 to that result, you get 22.' role='assistant' tool_calls=None
If you multiply 3 by 5, you get 15. If you then add 7 to that result, you get 22.


3. Create a data analyst

In [8]:
@tool
def get_games(num_games:int=1, top:bool=True) -> str:
    """
    Returns the top or bottom N games with highest or lowest scores.    
    args:
        num_games (int): Number of games to return (default is 1)
        top (bool): If True, return top games, otherwise return bottom (default is True)
    """
    data = [
        {"Game": "The Legend of Zelda: Breath of the Wild", "Platform": "Switch", "Score": 98},
        {"Game": "Super Mario Odyssey", "Platform": "Switch", "Score": 97},
        {"Game": "Metroid Prime", "Platform": "GameCube", "Score": 97},
        {"Game": "Super Smash Bros. Brawl", "Platform": "Wii", "Score": 93},
        {"Game": "Mario Kart 8 Deluxe", "Platform": "Switch", "Score": 92},
        {"Game": "Fire Emblem: Awakening", "Platform": "3DS", "Score": 92},
        {"Game": "Donkey Kong Country Returns", "Platform": "Wii", "Score": 87},
        {"Game": "Luigi's Mansion 3", "Platform": "Switch", "Score": 86},
        {"Game": "Pikmin 3", "Platform": "Wii U", "Score": 85},
        {"Game": "Animal Crossing: New Leaf", "Platform": "3DS", "Score": 88}
    ]
    # Sort the games list by Score
    # If top is True, descending order
    sorted_games = sorted(data, key=lambda x: x['Score'], reverse=top)
    
    # Return the N games
    return sorted_games[:num_games]

In [ ]:
# Create an agent with the multiple tools
data_analyst_agent = Agent(
    role="Game Stats Assistant",
    instructions="You can bring insights about a game dataset based on users questions",
    tools=[get_games]
)

In [10]:
response = data_analyst_agent.invoke("What's the best game in the dataset?")
print(response)

content="You're an AI Agent and your role is Game Stats Assistant. Your instructions: You can bring insights about a game dataset based on users questions" role='system'
content="What's the best game in the dataset?" role='user'
content=None role='assistant' tool_calls=[ChatCompletionMessageToolCall(id='call_lCcYDtEfEHiNd6jSGcv3HlvC', function=Function(arguments='{"num_games":1,"top":true}', name='get_games'), type='function')]
content='[{"Game": "The Legend of Zelda: Breath of the Wild", "Platform": "Switch", "Score": 98}]' role='tool' tool_call_id='call_lCcYDtEfEHiNd6jSGcv3HlvC' name='get_games'
content='The best game in the dataset is "The Legend of Zelda: Breath of the Wild" for the Switch, with a score of 98.' role='assistant' tool_calls=None
The best game in the dataset is "The Legend of Zelda: Breath of the Wild" for the Switch, with a score of 98.


In [11]:
response = data_analyst_agent.invoke("What's the worst game in the dataset?")
print(response)

content="You're an AI Agent and your role is Game Stats Assistant. Your instructions: You can bring insights about a game dataset based on users questions" role='system'
content="What's the worst game in the dataset?" role='user'
content=None role='assistant' tool_calls=[ChatCompletionMessageToolCall(id='call_2gOlDuDgH8YYAB0bRvV4YY5d', function=Function(arguments='{"num_games":1,"top":false}', name='get_games'), type='function')]
content='[{"Game": "Pikmin 3", "Platform": "Wii U", "Score": 85}]' role='tool' tool_call_id='call_2gOlDuDgH8YYAB0bRvV4YY5d' name='get_games'
content='The worst game in the dataset is "Pikmin 3" for the Wii U, with a score of 85.' role='assistant' tool_calls=None
The worst game in the dataset is "Pikmin 3" for the Wii U, with a score of 85.
